In [ ]:
!git clone https://github.com/huggingface/diffusers.git
!cd /content/diffusers && pip install .
!pip install transformers scipy ftfy accelerate
!pip install "ipywidgets>=7,<8"

Based on: https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb#scrollTo=AAVZStIokTVv

Documentation:
https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

# Checkout here which schedulers one can use out of the box:
# https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion#diffusers.StableDiffusionPipeline
scheduler = EulerDiscreteScheduler.from_pretrained("stabilityai/stable-diffusion-2-1-base", subfolder="scheduler")

# revision and dtype make sure that we use lower GPU memory but maybe sacrifice some of the quality
pipe = StableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base",
        scheduler=scheduler, 
        revision="fp16", 
        torch_dtype=torch.float16,
        requires_safety_checker=False
    )  
pipe = pipe.to("cuda")

: 

In [ ]:
prompt = "a photograph of an astronaut riding a black horse"

generator = torch.Generator("cuda").manual_seed(1024)

image = pipe(prompt, generator=generator).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

image

### Generating multiple outputs

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
num_images = 3
prompt = ["a photograph of an astronaut riding a horse"] * num_images

generator = torch.Generator("cuda").manual_seed(1024)

images = pipe(prompt, generator=generator, num_inference_steps=15).images

grid = image_grid(images, rows=1, cols=3)
grid

In [ ]:
generator = torch.Generator("cuda").manual_seed(1024)

images = pipe(prompt, generator=generator, num_inference_steps=5).images

grid = image_grid(images, rows=1, cols=3)
grid